元编程将程序当做数据，或在运行期完成编译期的工作。

装饰器类似于AOP编程范式，即面向切面编程，就是在不修改目标源码的前提下，添加功能的技术手段或设计模式，它是对OOP的补充。

In [1]:
def log(fn):
    def wraps(*args, **kwargs):
        print(*args, **kwargs)
        return fn(*args, **kwargs)
    return wraps

In [2]:
import dis

In [3]:
dis.dis(log)

  2           0 LOAD_CLOSURE             0 (fn)
              2 BUILD_TUPLE              1
              4 LOAD_CONST               1 (<code object wraps at 0x1079fc270, file "<ipython-input-1-bc3d2593a215>", line 2>)
              6 LOAD_CONST               2 ('log.<locals>.wraps')
              8 MAKE_FUNCTION            8
             10 STORE_FAST               1 (wraps)

  5          12 LOAD_FAST                1 (wraps)
             14 RETURN_VALUE


In [4]:
def add(x,y):
    return x+y

In [7]:
log(add)(1,4)

1 4


5

In [8]:
@log
def add(x,y):
    return x+y

In [9]:
dis.dis(add)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (args)
              4 LOAD_FAST                1 (kwargs)
              6 CALL_FUNCTION_EX         1
              8 POP_TOP

  4          10 LOAD_DEREF               0 (fn)
             12 LOAD_FAST                0 (args)
             14 LOAD_FAST                1 (kwargs)
             16 CALL_FUNCTION_EX         1
             18 RETURN_VALUE


In [10]:
def log(param):
    def decorator(fn):
        def wraps(*args, **kwargs):
            print(*args, **kwargs)
            return fn(*args, **kwargs)
        return wraps
    return decorator

In [11]:
dis.dis(log)

  2           0 LOAD_CONST               1 (<code object decorator at 0x107b13c90, file "<ipython-input-10-9f974b1647fa>", line 2>)
              2 LOAD_CONST               2 ('log.<locals>.decorator')
              4 MAKE_FUNCTION            0
              6 STORE_FAST               1 (decorator)

  7           8 LOAD_FAST                1 (decorator)
             10 RETURN_VALUE


In [12]:
@log('haha')
def add(x,y):
    return x+y

In [13]:
add(1,4)

1 4


5

In [16]:
def add(x,y):
    return x+y
log('haha')(add)(1,4)

1 4


5

类实现的装饰器用于实例方法时，会导致方法绑定丢失。

In [17]:
class log:
    def __init__(self, fn):
        self.fn = fn
    def __call__(self, *args, **kwargs):
        print('log ',*args, **kwargs)
        return self.fn(*args, **kwargs)

In [23]:
class X:
    @log
    def test(self):
        print(1+2)
        return 5

In [27]:
x = X()

In [20]:
x.test  # 方法被装饰器实例替代

In [21]:
x.test()

log 


TypeError: test() missing 1 required positional argument: 'self'

In [28]:
x.test(x)

log  <__main__.X object at 0x107b3c5c0>
3


5

因为装饰器实例替换了方法，结果导致实现绑定的描述符方法被隐藏，无法自动调用。要么让装饰器也实现描述符协议，要么显式传递参数。这样反而不如函数简单，因为函数默认实现了描述符协议和绑定规则，状态维持的话函数一样可以添加属性

多个装饰器嵌套时，我们需确保类型方法的装饰器是最外面的一个，因为无法确定内层装饰器的实现，可能也会因为描述符问题导致方法绑定失效。

In [ ]:
import functools
def log(fn):
    @functools.wraps(fn)
    def wrapped(*args, **kwargs):
        print(f'wrap:{id(wrapped)}, fn:{id(fn)}')
        return fn(*args, **kwargs)
    return wrapped

In [33]:
@log
def add(x:int, y:int) -> int:
    return x+y

In [34]:
add(1,6)

wrap:4424115200, fn:4424114792


7

In [35]:
add.__name__

'add'

In [36]:
add.__annotations__

{'return': int, 'x': int, 'y': int}

In [37]:
id(add)

4424115200

In [39]:
id(add.__wrapped__)

4424114792

functools.wraps会将原函数的\_\_module__, \_\_name__, \_\_doc__, \_\_annotations__等属性复制到包装函数，然后将原函数或上一装饰器保存在\_\_wrapped__中

In [41]:
def log(cls):
    
    class wrapper:
        def __init__(self, *args, **kwargs):
            self.__dict__['inst'] = cls(*args, **kwargs)
        def __getattr__(self, name):
            value = getattr(self.inst, name)
            print('get value ', value)
            return value
        def __setattr__(self, name, value):
            print('set value', value)
            return setattr(self.inst, name, value)
    return wrapper

In [42]:
@log
class X:
    pass

In [43]:
x = X()

In [44]:
x.a = 5

set value 5


In [45]:
x.a

get value  5


5

用于类别的装饰器，也可以用函数来写

In [46]:
def log(cls):
    def wrapper(*args, **kwargs):
        o = cls(*args, **kwargs)
        print('log ',o)
        return o
    return wrapper
@log
class X:
    pass

In [47]:
X()

log  <__main__.X object at 0x107b461d0>


调用跟踪

In [48]:
def call_count(fn):
    def counter(*args, **kwargs):
        counter.__count__ += 1
        return fn(*args, **kwargs)
    counter.__count__ = 0
    return counter
@call_count
def a():
    pass
@call_count
def b():
    pass

In [49]:
a()
a()
a.__count__

2

In [50]:
b()
b()
b()
b.__count__

3

In [55]:
from time import sleep
@functools.lru_cache(10)
def test(x):
    sleep(x)

In [57]:
%%time
for i in range(1000):
    test(1)

CPU times: user 126 µs, sys: 1e+03 ns, total: 127 µs
Wall time: 130 µs


属性管理

In [58]:
def pet(cls):
    cls.dosth = lambda self: None
    return cls
@pet
class Parrot:
    pass

实例管理

In [59]:
def singleton(cls):
    inst = None
    def wrap(*args, **kwargs):
        nonlocal inst
        if not inst:
            inst = cls(*args, **kwargs)
        return inst
    return wrap

In [60]:
@singleton
class X:
    pass

In [61]:
X() is X()

True

部件注册

In [62]:
class App:
    def __init__(self):
        self.routers = {}
    def route(self, url):
        def register(fn):
            self.routers[url] = fn
            return fn
        return register

In [64]:
app =App()
@app.route('/')
def home():
    pass
@app.route('/help')
def help():
    pass


In [65]:
app.routers

{'/': <function __main__.home>, '/help': <function __main__.help>}

描述符不同于实例的通用拦截方法(\_\_getattr__等)，它以单个属性出现，并针对该属性的不同访问行为自动做出响应。

In [120]:
class descriptor:
    def __set_name__(self, owner, name):
        print(f'name:{owner.__name__},,{name}')
        self.name = f"__{name}__"
    def __get__(self, instance, owner):
        print(f'get:{instance}, {owner}')
        return getattr(instance, self.name, None)
    def __set__(self, instance, value):
        print(f'set:{instance}, {value}')
        return setattr(instance, self.name, value)
    def __delete__(self, instance):
        print(f'delete: {instance}')
        raise AttributeError('delete is disabled')

In [131]:
class X:
    data = descriptor()
    print(data.__get__)

<bound method descriptor.__get__ of <__main__.descriptor object at 0x107b9fcf8>>
name:X,,data


创建属性时，\_\_set_name__方法被调用，并可通过参数获知目标类型以及属性名称。

In [122]:
x = X()

In [123]:
x.data = 500

set:<__main__.X object at 0x107bafac8>, 500


In [124]:
x.data

get:<__main__.X object at 0x107bafac8>, <class '__main__.X'>


500

In [125]:
del x.data

delete: <__main__.X object at 0x107bafac8>


AttributeError: delete is disabled

In [126]:
X.data

get:None, <class '__main__.X'>


In [76]:
X.data = 99

In [77]:
X.data

99

以类型或实例访问描述符属性时，\_\_get__被自动调用，且会接收到类型和实例的引用。  
\_\_set__和\_\_delete__仅在实例引用时被调用，以类型引用时会导致描述符属性被替换或删除。  
将描述符属性赋值给变量或传参时，实际结果是\_\_get__方法的返回值。

定义了\_\_set__或\_\_delete__则为数据描述符，仅有\_\_get__为非数据描述符，访问顺序是：数据描述符，\_\_dict__，非数据描述符。

In [128]:
p=property()
p.__get__

<method-wrapper '__get__' of property object at 0x107b97408>

In [129]:
p.__set__

<method-wrapper '__set__' of property object at 0x107b97408>

In [130]:
p.__delete__

<method-wrapper '__delete__' of property object at 0x107b97408>

函数默认实现了描述符协议，所以当以实例或类型访问方法时，\_\_get__首先被调用。类型和实例做为参数被传入\_\_get__，从而截获绑定目标（\_\_self__），如此就将函数包装成绑定方法对象返回。实际被执行的，就是这个会隐式传入第一参数的包装品。

In [139]:
class X:
    def test(self, o):
        print(o)

In [141]:
x = X()

In [142]:
x.test

<bound method X.test of <__main__.X object at 0x107b9de10>>

In [143]:
m = x.test.__get__(x, X)

In [144]:
m

<bound method X.test of <__main__.X object at 0x107b9de10>>

In [145]:
m.__self__, m.__func__

(<__main__.X at 0x107b9de10>, <function __main__.X.test>)

In [146]:
m(123)

123


方法执行实际上分为了两个步骤：  
x.test(123) 相当于 m = x.test.\_\_get__(x, type(x)), m(123)，首先将函数包装为绑定方法。  
X.test(m.\_\_self__, 123)，然后执行时隐式将self/cls参数传给目标函数。  
在绑定方法对象内，\_\_self__和\_\_func__存储了执行所需的信息。

In [149]:
x.test.__get__

<method-wrapper '__get__' of method object at 0x1079e1108>